In [1]:
# Import Libraries
import numpy as np
import pandas as pd
from keras.optimizers import *
from keras.models import Sequential
from keras.layers import Dense,Conv1D,Dropout,MaxPooling1D,Flatten,Reshape
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedShuffleSplit
from IPython.display import HTML, display
import tabulate

Using TensorFlow backend.


## Leer Dataset

Leemos el dataset correspondiente: 

- data_windows: Es el dataset con vectores de 528 características
- data_windows_264: Es el dataset con vectores de 264 características
- data_windows_132: Es el dataset con vectores de 132 características

In [2]:
#Dataset
dataset = pd.read_csv('dataset/data_windows_132.csv').iloc[:,1:]

In [3]:
dataset.shape

(29848, 133)

## Paso se preprocesamiento

Se eliminan vectores que contentan datos Nan.

In [4]:
#Delete NAN values
"""
dataset_x = []
dataset_y = []
for label in range(1,17):
    data = dataset[dataset.iloc[:,0] == label]
    for index, row in data.iterrows():
        if (len(dataset_x) <= (26*label)):
            dataset_x.append(np.nan_to_num(dataset.iloc[index,1:].values))
            dataset_y.append(dataset.iloc[index,0:1].values)
        else:
            break
"""
dataset_x = []
dataset_y = []

count = np.zeros(17)

for label in range(1,17):
    data = dataset[dataset.iloc[:,0] == label]
    for index, row in data.iterrows():
        if (len(dataset_x) <= (810*label)):
            dataset_x.append(np.nan_to_num(dataset.iloc[index,1:].values))
            dataset_y.append(dataset.iloc[index,0:1].values)
            count[label]+=1
    
dataset_x = np.array(dataset_x)
dataset_y = np.array(dataset_y)

In [5]:
dataset_x.shape,dataset_y.shape

((12961, 132), (12961, 1))

In [6]:
print(count[1:])

[811. 810. 810. 810. 810. 810. 810. 810. 810. 810. 810. 810. 810. 810.
 810. 810.]


In [7]:
#Normalice X values
mean = np.sum(dataset_x,axis=0,keepdims=True)/dataset_x.shape[0]
normal_variance = np.sum(dataset_x**2,axis=0,keepdims=True)/dataset_x.shape[0]
dataset_x_normalice = dataset_x-mean
dataset_x_normalice = dataset_x_normalice/np.sqrt(normal_variance+1e-8)

In [8]:
dataset_y_reshape = dataset_y.reshape(1,dataset_y.shape[0]).flatten().astype(int)
dataset_y_reshape[20:40]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [9]:
#Create Y binary class vector
y_train_vector = dataset_y_reshape
print(y_train_vector[0])
print(y_train_vector.shape)

1
(12961,)


## Metodo Test Configuration

El método admite un vector de valores para cada hiperparámetro a probar y ejecuta la prueba con 4 KFolds para cada configuración posible, finalmente imprime los resultados en una tabla.

In [75]:
def test_configuration(lr_v = [0.001], beta_1_v=[0.9],beta_2_v=[0.999] ,epochs_n = [10], batch_size_n = [32], input_shape_n = 1056):
    
    configuration = 1
    table = [["Config","Train 1","Test 1","Train 2","Test 2","Train 3","Test 3","Train 4","Test 4","Train Prom","Test Prom"]]
    
    for batch in batch_size_n:
        for epoch in epochs_n:
            for beta2 in beta_2_v:
                for beta1 in beta_1_v:
                    for lr in lr_v:
                        
                        #Dividimos en train y test, ramdon_state es una semilla para generar números aleatorios
                        sss =  StratifiedShuffleSplit(n_splits=4,test_size=0.2, random_state=1)
                        #kf.get_n_splits(dataset_x_normalice)
                        sss.get_n_splits(dataset_x_normalice, y_train_vector)
                        index_kfol = []
                        for train_index, test_index in sss.split(dataset_x_normalice, y_train_vector):
                            index_kfol.append([train_index,test_index])
                        
                        #For save accuracies
                        results = {}
                        
                        # Create Model
                        model = Sequential([
                            Reshape((12, 11), input_shape=(132,)),
                            Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(12,11)),
                            Conv1D(filters=64, kernel_size=2, activation='relu'),
                            Dropout(0.5),
                            MaxPooling1D(pool_size=2),
                            Flatten(),                            
                            Dense(128, activation='relu'),
                            Dense(32, activation='relu'),
                            Dense(16, activation='softmax'),
                        ])

                        #Compile Model
                        model.compile(
                          optimizer=Adam(learning_rate=lr, beta_1=beta1, beta_2=beta2, amsgrad=False),
                          loss='categorical_crossentropy',
                          metrics=['accuracy'],
                        )
                        
                        Wsave = model.get_weights()

                        for g in range(4):

                            model.set_weights(Wsave)

                            #Get K-fold group 
                            train_index = index_kfol[g][0]
                            test_index = index_kfol[g][1]

                            x_train = dataset_x_normalice[train_index]
                            y_train = y_train_vector[train_index]
                            x_test = dataset_x_normalice[test_index]
                            y_test = y_train_vector[test_index]  

                            history = model.fit(
                              x_train, # training data
                              to_categorical(y_train-1), # training targets
                              epochs=epoch,
                              batch_size=batch,
                            )

                            results[str(g)+"train"] = history.history['accuracy'][-1]

                            output = model.evaluate(x_test,to_categorical(y_test-1))

                            results[str(g)+"test"] = output[1]
                            
                        #Print Results
                        print("______________________Configuration-"+str(configuration)+"______________________")
                        print("LR:"+str(lr)+"- Beta_1:"+str(beta1)+"- Beta_2:"+str(beta2))
                        print("Epochs:"+str(epoch)+"- Bach:"+str(batch))
                        print("_________________________________________________________")
                        
                        prom_train = []
                        prom_test = []
                        for g in range(4):
                            prom_train.append(results[str(g)+"train"]*100)
                            prom_test.append(results[str(g)+"test"]*100)
                        
                        table.append([configuration,
                                      "%.2f" % round(results["0train"]*100,2),"%.2f" % round(results["0test"]*100,2),
                                      "%.2f" % round(results["1train"]*100,2),"%.2f" % round(results["1test"]*100,2),
                                      "%.2f" % round(results["2train"]*100,2),"%.2f" % round(results["2test"]*100,2),
                                      "%.2f" % round(results["3train"]*100,2),"%.2f" % round(results["3test"]*100,2),
                                      "%.2f" % round(sum(prom_train)/4,2),"%.2f" % round(sum(prom_test)/4,2)])
                        print("_______________________ENDTEST_________________________")   
                        configuration = configuration+1
                        
    display(HTML(tabulate.tabulate(table, tablefmt='html')))                                      
                        

In [78]:
lr_t=[0.00094]
beta_1_t = [0.9]
beta_2_t = [0.999]
epochs_t = [20]
batch_size_t = [16]
test_configuration(lr_v=lr_t,beta_1_v=beta_1_t,beta_2_v=beta_2_t,epochs_n = epochs_t,batch_size_n = batch_size_t)

Epoch 1/20
10368/10368 [==============================] - 2s 151us/step - loss: 0.7918 - accuracy: 0.7587
Epoch 2/20
10368/10368 [==============================] - 1s 137us/step - loss: 0.3180 - accuracy: 0.9079
Epoch 3/20
10368/10368 [==============================] - 1s 137us/step - loss: 0.2312 - accuracy: 0.9305
Epoch 4/20
10368/10368 [==============================] - 1s 139us/step - loss: 0.2041 - accuracy: 0.9372
Epoch 5/20
10368/10368 [==============================] - 1s 135us/step - loss: 0.1765 - accuracy: 0.9448
Epoch 6/20
10368/10368 [==============================] - 1s 139us/step - loss: 0.1602 - accuracy: 0.9473
Epoch 7/20
10368/10368 [==============================] - 1s 139us/step - loss: 0.1535 - accuracy: 0.9486
Epoch 8/20
10368/10368 [==============================] - 1s 137us/step - loss: 0.1329 - accuracy: 0.9556
Epoch 9/20
10368/10368 [==============================] - 1s 136us/step - loss: 0.1282 - accuracy: 0.9569
Epoch 10/20
10368/10368 [=====================

Config,Train 1,Test 1,Train 2,Test 2,Train 3,Test 3,Train 4,Test 4,Train Prom,Test Prom
1,97.41,96.03,97.39,96.14,97.53,96.88,97.38,96.03,97.42,96.27


Epoch 1/20
10368/10368 [==============================] - 2s 153us/step - loss: 0.7674 - accuracy: 0.7685
Epoch 2/20
10368/10368 [==============================] - 1s 137us/step - loss: 0.3145 - accuracy: 0.9024
Epoch 3/20
10368/10368 [==============================] - 1s 136us/step - loss: 0.2399 - accuracy: 0.9248
Epoch 4/20
10368/10368 [==============================] - 1s 144us/step - loss: 0.2011 - accuracy: 0.9343
Epoch 5/20
10368/10368 [==============================] - 1s 139us/step - loss: 0.1748 - accuracy: 0.9426
Epoch 6/20
10368/10368 [==============================] - 1s 137us/step - loss: 0.1654 - accuracy: 0.9419
Epoch 7/20
10368/10368 [==============================] - 1s 136us/step - loss: 0.1454 - accuracy: 0.9533
Epoch 8/20
10368/10368 [==============================] - 1s 135us/step - loss: 0.1315 - accuracy: 0.9558
Epoch 9/20
10368/10368 [==============================] - 1s 134us/step - loss: 0.1369 - accuracy: 0.9545
Epoch 10/20
10368/10368 [=====================

Config,Train 1,Test 1,Train 2,Test 2,Train 3,Test 3,Train 4,Test 4,Train Prom,Test Prom
1,97.39,96.68,97.52,96.53,97.51,97.34,97.09,96.72,97.38,96.82


Epoch 1/20
10368/10368 [==============================] - 2s 147us/step - loss: 0.7845 - accuracy: 0.7642
Epoch 2/20
10368/10368 [==============================] - 1s 137us/step - loss: 0.3360 - accuracy: 0.8984
Epoch 3/20
10368/10368 [==============================] - 1s 135us/step - loss: 0.2570 - accuracy: 0.9197
Epoch 4/20
10368/10368 [==============================] - 1s 134us/step - loss: 0.2074 - accuracy: 0.9345
Epoch 5/20
10368/10368 [==============================] - 1s 134us/step - loss: 0.1872 - accuracy: 0.9407
Epoch 6/20
10368/10368 [==============================] - 1s 135us/step - loss: 0.1649 - accuracy: 0.9484
Epoch 7/20
10368/10368 [==============================] - 1s 136us/step - loss: 0.1486 - accuracy: 0.9531
Epoch 8/20
10368/10368 [==============================] - 1s 137us/step - loss: 0.1436 - accuracy: 0.9539
Epoch 9/20
10368/10368 [==============================] - 1s 136us/step - loss: 0.1380 - accuracy: 0.9537
Epoch 10/20
10368/10368 [=====================

...

Config,Train 1,Test 1,Train 2,Test 2,Train 3,Test 3,Train 4,Test 4,Train Prom,Test Prom
1,97.42,95.87,97.78,96.80,97.08,96.57,97.34,96.80,97.40,96.51


Epoch 1/20
10368/10368 [==============================] - 2s 146us/step - loss: 0.8389 - accuracy: 0.7496
Epoch 2/20
10368/10368 [==============================] - 1s 131us/step - loss: 0.3170 - accuracy: 0.9050
Epoch 3/20
10368/10368 [==============================] - 1s 132us/step - loss: 0.2397 - accuracy: 0.9267
Epoch 4/20
10368/10368 [==============================] - 1s 133us/step - loss: 0.1918 - accuracy: 0.9387
Epoch 5/20
10368/10368 [==============================] - 1s 131us/step - loss: 0.1735 - accuracy: 0.9445
Epoch 6/20
10368/10368 [==============================] - 1s 131us/step - loss: 0.1602 - accuracy: 0.9495
Epoch 7/20
10368/10368 [==============================] - 1s 131us/step - loss: 0.1344 - accuracy: 0.9569
Epoch 8/20
10368/10368 [==============================] - 1s 131us/step - loss: 0.1325 - accuracy: 0.9556
Epoch 9/20
10368/10368 [==============================] - 1s 130us/step - loss: 0.1246 - accuracy: 0.9579
Epoch 10/20
10368/10368 [=====================

Config,Train 1,Test 1,Train 2,Test 2,Train 3,Test 3,Train 4,Test 4,Train Prom,Test Prom
1,97.63,96.76,97.12,96.53,97.31,96.57,97.36,95.83,97.35,96.42


Epoch 1/20
10368/10368 [==============================] - 1s 136us/step - loss: 0.8068 - accuracy: 0.7620
Epoch 2/20
10368/10368 [==============================] - 1s 123us/step - loss: 0.3075 - accuracy: 0.9058
Epoch 3/20
10368/10368 [==============================] - 1s 126us/step - loss: 0.2358 - accuracy: 0.9280
Epoch 4/20
10368/10368 [==============================] - 1s 125us/step - loss: 0.1958 - accuracy: 0.9399
Epoch 5/20
10368/10368 [==============================] - 1s 122us/step - loss: 0.1766 - accuracy: 0.9450
Epoch 6/20
10368/10368 [==============================] - 1s 122us/step - loss: 0.1475 - accuracy: 0.9524
Epoch 7/20
10368/10368 [==============================] - 1s 122us/step - loss: 0.1392 - accuracy: 0.9531
Epoch 8/20
10368/10368 [==============================] - 1s 122us/step - loss: 0.1277 - accuracy: 0.9579
Epoch 9/20
10368/10368 [==============================] - 1s 122us/step - loss: 0.1221 - accuracy: 0.9595
Epoch 10/20
10368/10368 [=====================

Config,Train 1,Test 1,Train 2,Test 2,Train 3,Test 3,Train 4,Test 4,Train Prom,Test Prom
1,97.41,96.30,97.46,96.61,97.50,96.41,97.69,95.68,97.52,96.25
